In [1]:
# First, import necessary packages
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt

### The point of this exercise is to investigate the issue with no residential properties being listed as being built after 2013

**As I go through ensure that I check both the 'Admin_ACTYEARBLT' and 'Admin_EFFYEARBLT'**

In [2]:
# Read the csv with all parcels
parcels = pd.read_csv('Admin.csv')

In [53]:
# What are the column names so we can use for creating sub-arrays
parcels.columns

Index(['SCHEDULE,C,7', 'PIN,C,12', 'MAPNUM,C,4', 'SEC_QUAD,C,3', 'BLK,C,2',
       'PARCELNB,C,3', 'OWNER,C,40', 'CAREOF,C,40', 'MAILINGADD,C,40',
       'CITY,C,40', 'STATE,C,2', 'ZIPCODE,C,10', 'IMPACT,N,19,11',
       'IMPASD,N,19,11', 'ACTLANDVAL,N,19,11', 'ASDLANDVAL,N,19,11',
       'AREAID,C,6', 'SUBDIVCODE,C,4', 'SUBNAME,C,40', 'BLOCK,C,5', 'LOT,C,5',
       'ANDTHRU,C,1', 'ANDLOT,C,5', 'CONDOCODE,C,4', 'CONNAME,C,40',
       'CONDOUNIT,C,6', 'LEGAL_NC,C,100', 'USE_CODE,C,6', 'UC_DESC,C,40',
       'LAND_CODE,C,6', 'ACTYEARBLT,C,4', 'EFFYEARBLT,C,4', 'BATHS,C,5',
       'BEDROOMS,C,5', 'STORIES,C,5', 'ARCH_STYLE,C,5', 'HEAT_TYPE,C,5',
       'HEAT_FUEL,C,5', 'CONST_QUAL,C,5', 'CONDITION,C,5',
       'HEATEDAREA,N,19,11', 'LANDAREA,N,19,11', 'NEIGHBORHD,N,19,11',
       'SALEDATE,D', 'DEEDTYPE,C,4', 'SALEPRICE,N,19,11', 'UPDATEDATE,C,10'],
      dtype='object')

In [54]:
x = parcels['SUBNAME,C,40'].value_counts()
x[x > 1].head()

METES & BOUNDS 76 ALL    858
INNSBRUCK-VAL MORITZ     569
COLUMBINE LAKE           544
METES & BOUNDS 75 ALL    474
GRAND LAKE               382
Name: SUBNAME,C,40, dtype: int64

In [55]:
parcels['SUBNAME,C,40'][0]

'METES & BOUNDS 75 ALL'

In [67]:
filter = parcels['SUBNAME,C,40'].dropna() #str.contains('Rendevous', regex=False)
filter1 = filter.str.contains('GRAND PARK', regex=False)

In [69]:
x = parcels.where(filter1)
x['SUBNAME,C,40'].value_counts()

COZENS POINTE AT GRAND PARK                 134
COZENS MEADOW AT GRAND PARK AMEND NO 1       61
MEADOWS AT GRAND PARK FILING 1               43
ELK CREEK AT GRAND PARK FILING 4             43
ELK CREEK AT GRAND PARK FILING 3             28
WILLOWS AT GRAND PARK                        26
WILLOWS AT GRAND PARK FILING 3               26
ELK CREEK AT GRAND PARK FILING 1             22
WILLOWS AT GRAND PARK FILING 2               20
ELK CREEK AT GRAND PARK FILING 2             18
MOOSE HOLLOW AT GRAND PARK PHASE 1            9
VILLAGE AT GRAND PARK FILING 1 (GENERAL*      8
GRAND PARK DRIVE EXEMPTION                    3
ELK CREEK CONDOMINIUMS AT GRAND PARK          2
VILLAGE AT GRAND PARK FILING 2                2
GRAND PARK SUBDIVISION                        1
VILLAGE AT GRAND PARK FILING 2A               1
VILLAGE AT GRAND PARK FILING 2 LOT 12C        1
Name: SUBNAME,C,40, dtype: int64

From the information above, let's do a similar analysis to see how many residential parcels have a "EFFYEARBLT" after 2010.

In [4]:
x = parcels[parcels['EFFYEARBLT,C,4'] > 2010]
len(x)

535

So, there are 535 parcels with an "EFFYEARBLT" greater than 2010. What is the max?

In [5]:
x['EFFYEARBLT,C,4'].max()

2019.0

So, there are parcels that have an "EFFYEARBLT" of greater than 2010 and all the way up to 2019, the last year we would expect. 

How about specific to residential, which is where our problem arose in the first place?

In [6]:
res = x[x['UC_DESC,C,40'] == 'Residential Improved']

In [7]:
len(res)

179

We have 179 residential improved parcels constructed after 2010. This is in alignment with my finding, somewhere else, that it seems about 60 homes are being built per year, if 2013 is the max year. 

In [8]:
res['EFFYEARBLT,C,4'].max()

2013.0

This still shows 2013 as being the max year for 'EFFYEARBLT'.

Just to assuage the woman's potential pushback, let us try 'ACTYEARBLT', even though many of the parcels have a "0" for this data, even when there is an "EFFYEARBLT". 



In [9]:
res['ACTYEARBLT,C,4'].max()

0.0

This is in alignment with what I saw going manually through the dataset, in that there was no "ACTYEARBLT" for any "Residential Improved"

So, if "Residential Improved" does not have anything past 2013, what does? We know condos do. about the others? 

In [10]:
# Get different land use types
y = parcels['UC_DESC,C,40']
uc = y.unique()

for i in uc:
    z = x[x['UC_DESC,C,40'] == i]
    print(i, z['ACTYEARBLT,C,4'].max())

nan nan
Residential Vacant 0.0
Residential Improved 0.0
Improved Metes & Bounds 0.0
Tax Exempt 2014.0
Vacant Land Metes & Bounds 0.0
Agricultural 0.0
Multi-Comm-Res-Ag-MH 2017.0
Condominium Improved 2018.0
Mobile Home/Res Metes & Bounds nan
Ag Odyssey 0.0
Mobile Home Land 0.0
Commercial Improved 2018.0
Commercial Vacant nan
Multi Units 9 & Up 0.0
Conservation Easement 0.0
Improvements Only 2012.0
Multi Units 4-8 nan
Mobile Homes Only 0.0
Forest Ag nan
Duplex-Triplex 0.0
Industrial Improved nan
Mother-In-Law Duplex nan
Condominium Vacant nan
Industrial Vacant nan
Severed Mineral Interst nan
Cemetery nan
Patented Mining Claim nan
Vacant land Metes & Bounds nan


As we see here, only Condo Improved (2018), Improvements only (2012), and Multi-Comm-Res... (2017) have any value for "ACTYEARBLT". 

Let's do the same but for "EFFYEARBLT".


In [61]:
for i in uc:
    aa = x[x['UC_DESC,C,40'] == i]
    print(i, aa['EFFYEARBLT,C,4'].max())

nan nan
Residential Vacant 2013.0
Residential Improved 2013.0
Improved Metes & Bounds 2013.0
Tax Exempt 2014.0
Vacant Land Metes & Bounds 2013.0
Agricultural 2013.0
Multi-Comm-Res-Ag-MH 2017.0
Condominium Improved 2018.0
Mobile Home/Res Metes & Bounds nan
Ag Odyssey 2011.0
Mobile Home Land 2012.0
Commercial Improved 2018.0
Commercial Vacant nan
Multi Units 9 & Up 2013.0
Conservation Easement 2012.0
Improvements Only 2012.0
Multi Units 4-8 nan
Mobile Homes Only 2012.0
Forest Ag nan
Duplex-Triplex 2013.0
Industrial Improved nan
Mother-In-Law Duplex nan
Condominium Vacant nan
Industrial Vacant nan
Severed Mineral Interst nan
Cemetery nan
Patented Mining Claim nan
Vacant land Metes & Bounds nan


We can see here that almost everything maxes out at 2013 or 2012. Commercial Improved and Condo Improved being the most recent. The issue is that none of these reach 2019... This is likely because of the Use_code, instead of the UC_Desc.

Let's try that... 

In [64]:
# Get different land use types by 'USE_CODE', instead of the description
holder = parcels['USE_CODE,C,6']
user_codes = holder.unique()
user_codes

array([   0., 1111., 1000.,  501.,   nan, 9000.,  500., 4000., 7500.,
       4002., 1100., 1300., 4001., 1200., 2000., 2111., 1225., 4003.,
        100., 2100., 1220.,  150., 1215., 3000., 1214., 1101., 3111.,
       5170., 1145., 5140.])

In [69]:
for i in user_codes:
    result = parcels[parcels['USE_CODE,C,6'] == i]
    print(i, result['EFFYEARBLT,C,4'].max())


0.0 0.0
1111.0 2013.0
1000.0 2013.0
501.0 2013.0
nan nan
9000.0 2014.0
500.0 2013.0
4000.0 2013.0
7500.0 2017.0
4002.0 2006.0
1100.0 2019.0
1300.0 2008.0
4001.0 2011.0
1200.0 2012.0
2000.0 2019.0
2111.0 2009.0
1225.0 2013.0
4003.0 2012.0
100.0 2012.0
2100.0 1997.0
1220.0 1999.0
150.0 2012.0
1215.0 2013.0
3000.0 2010.0
1214.0 2005.0
1101.0 2009.0
3111.0 0.0
5170.0 0.0
1145.0 0.0
5140.0 0.0
